In [ ]:
# 3015 오아시스 재결합 (P5)

from collections import deque
stack = deque() # 같거나 작은 순서대로 정렬할 스택
lst = deque() # 스택에 해당하는 값에서 앞에 해당하는 사람들과 마주볼 수 있는 횟수

n = int(input())
cnt = 0 # 총 횟수
stack.append(int(input()))
stack_size = 1
lst.append(0) # 앞에서 사람이 없으므로 0

for i in range(1, n):
    temp = int(input())

    while stack and stack[-1] < temp: # 들어올려는 숫자가 크다면 그냥 더하는 것이 아닌 갱신
        stack.pop()
        cnt += lst.pop() + 1 # 도중에 스택에서 제거하려면 lst 에 해당하는 값 + 다음 사람과 만날 횟수(1)를 cnt 에 추가해야만 한다.
        stack_size -= 1

    if stack_size > 0:
        if stack[-1] > temp: # 들어가려는 수가 스택의 마지막보다 작다면 앞선 사람 1명과 만날 수 있다.
            lst.append(1)
        else:
            lst.append(lst[-1] + 1) # 만약에 값이 같다면 앞선 사람에 해당하는 값 + 1 명과 만날 수 있다.
    else:
        lst.append(0) # 앞선 사람과 만날 수 없는 경우에는 0이다.

    stack.append(temp)
    stack_size += 1

print(cnt + sum(lst)) # 최종적으로 계산된 값

In [ ]:
# 13537 수열과 쿼리 1
# https://www.acmicpc.net/problem/13537

# 머지 소트 트리에 대해서 알아야 풀 수 있는 문제
# 기본적으로 메모리 소모가 매우 큰 것으로 파악되며, 기본 sort를 사용하는 것이 더 빠르다.
# 속도를 더 올리고 싶다면 python bisect(배열 이진 분할 알고리즘)에 대해서 알아보도록 하자
# https://docs.python.org/ko/3/library/bisect.html

# def merge_sort(size, A, B):
#     ret = A + B
#     ret.sort()
#
#     return ret


def merge_sort(size, A, B):
    ret = [0] * size
    child_size = size // 2

    pt1 = 0
    pt2 = 0
    idx = 0

    while pt1 < child_size and pt2 < child_size:
        if A[pt1] <= B[pt2]:
            ret[idx] = A[pt1]
            pt1 += 1
        else:
            ret[idx] = B[pt2]
            pt2 += 1

        idx += 1

    while pt1 < child_size:
        ret[idx] = A[pt1]
        pt1 += 1
        idx += 1

    while pt2 < child_size:
        ret[idx] = B[pt2]
        pt2 += 1
        idx += 1

    return ret


def binary_search(size, lst, val):

    if lst[-1] <= val:
        return 0

    start = 0
    end = size - 1

    while start < end:
        mid = (start + end) // 2

        if lst[mid] <= val:
            start = mid + 1
        else:
            end = mid

    return size - start


def segment(s, e, val):
    global tree, tree_size

    ret = 0
    while s < e:
        if s % 2 == 1:
            ret += binary_search(tree_size[s], tree[s], val)
        if e % 2 == 0:
            ret += binary_search(tree_size[e], tree[e], val)

        s = (s + 1) // 2
        e = (e - 1) // 2

    if s == e:
        ret += binary_search(tree_size[s], tree[s], val)

    return ret


n = int(input())
lst = tuple(map(int, input().split()))
depth = 1
gap = 1

while n > gap:
    depth += 1
    gap <<= 1

tree = [0] * (gap * 2)
tree_size = [0] * (gap * 2)
tree_size[1] = gap

for i in range(1, gap):
    temp = tree_size[i] // 2
    tree_size[2 * i] = temp
    tree_size[2 * i + 1] = temp

for i in range(n):
    tree[gap + i] = [lst[i]]

for i in range(n, gap):
    tree[gap + i] = [0]

for i in range(gap - 1, 0, -1):
    left_child = tree[2 * i]
    right_child = tree[2 * i + 1]
    tree[i] = merge_sort(tree_size[i], left_child, right_child)

m = int(input())
for i in range(m):
    a, b, c = map(int, input().split())
    a += (gap - 1)
    b += (gap - 1)
    print(segment(a, b, c))